In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from printer_anomaly_detection.dataset.audio import get_audio_duration, get_recording_time
from pathlib import Path

2023-06-20 23:03:13.377158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 23:03:14.022539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
dataset_path = Path('../../../../datasets/prints/')
assert dataset_path.exists()

In [5]:
from collections import namedtuple
from dataclasses import dataclass
from typing import List
import dataclass_csv
from printer_anomaly_detection.dataset.audio import Upgrade, get_audio_dataset_files, get_recording_time
from printer_anomaly_detection.dataset.domain import Datetime
from printer_anomaly_detection.preprocessing.data import Outcome


with open(dataset_path / 'upgrades.csv', 'r', encoding='utf-8') as f:
    upgrades = list(sorted(dataclass_csv.DataclassReader(f, Upgrade), key= lambda u: u.when))

FileDurationPair = namedtuple('FileDurationPair', ['file', 'duration', 'recording_time'])

@dataclass
class DatasetConfig:
    after: Datetime
    before: Datetime
    name: str
    files: List[FileDurationPair]

    @property
    def duration(self):
        return sum(map(lambda f: f.duration, self.files))

configs = []

last_upgrade = 'original'
after = Datetime.min
for upgrade in upgrades + [Upgrade('nothing', Datetime.max)]:
    if 'audio' not in upgrade.effects:
        continue
    before = upgrade.when
    number_of_successes = len(list(get_audio_dataset_files(dataset_path, after, before, {Outcome.SUCCESS})))
    number_of_failures = len(list(get_audio_dataset_files(dataset_path, after, before, {Outcome.FAILURE})))

    success_files = list(get_audio_dataset_files(dataset_path, after, before, {Outcome.SUCCESS}))
    success_file_duration_pairs = [FileDurationPair(file.parent.name, get_audio_duration(file), get_recording_time(file.parent)) for file in success_files]
    
    configs.append(
        DatasetConfig(
            after=after,
            before=before,
            name=last_upgrade,
            files=success_file_duration_pairs
        )
    )
    after = before
    last_upgrade = upgrade.what


for config in configs:
    print(config)

/home/y2g/projects/personal/printer-anomaly-detection/src/printer_anomaly_detection/dataset/audio.py:30: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  return librosa.get_duration(filename=path.as_posix())
/home/y2g/projects/personal/printer-anomaly-detection/src/printer_anomaly_detection/dataset/audio.py:30: FutureWarning: PySoundFile failed. Trying audioread instead.
	Audioread support is deprecated in librosa 0.10.0 and will be removed in version 1.0.
  return librosa.get_duration(filename=path.as_posix())
/home/y2g/projects/personal/printer-anomaly-detection/src/printer_anomaly_detection/dataset/audio.py:30: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  return librosa.get_duration(filename=path.as_posix())
/home/y2g/projects/personal/printer-anomaly-detection/src/printer_anomaly

DatasetConfig(after=0001-01-01T00:00:00, before=2023-04-21T10:00:00, name='original', files=[FileDurationPair(file='logitech_cam_mount', duration=2187.4, recording_time=2023-04-18T20:09:44.973695), FileDurationPair(file='arducam_case_mount', duration=6096.3, recording_time=2023-04-18T18:04:26.302213), FileDurationPair(file='test', duration=11.8, recording_time=2023-04-17T19:23:34.904896), FileDurationPair(file='cable_clips', duration=5403.6, recording_time=2023-04-16T13:38:57.167432), FileDurationPair(file='cable_clips2', duration=3153.0, recording_time=2023-04-17T19:29:53.188516)])
DatasetConfig(after=2023-04-21T10:00:00, before=2023-05-30T12:00:00, name='exhaust', files=[FileDurationPair(file='adj_earplug', duration=11160.7, recording_time=2023-04-24T17:33:30.833910), FileDurationPair(file='hinge_small', duration=6483.9, recording_time=2023-04-25T19:22:31.792299), FileDurationPair(file='lily_58_frame', duration=2607.5, recording_time=2023-04-26T18:21:01.485980), FileDurationPair(file

/home/y2g/projects/personal/printer-anomaly-detection/src/printer_anomaly_detection/dataset/audio.py:30: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  return librosa.get_duration(filename=path.as_posix())
/home/y2g/projects/personal/printer-anomaly-detection/src/printer_anomaly_detection/dataset/audio.py:30: FutureWarning: PySoundFile failed. Trying audioread instead.
	Audioread support is deprecated in librosa 0.10.0 and will be removed in version 1.0.
  return librosa.get_duration(filename=path.as_posix())
/home/y2g/projects/personal/printer-anomaly-detection/src/printer_anomaly_detection/dataset/audio.py:30: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  return librosa.get_duration(filename=path.as_posix())
/home/y2g/projects/personal/printer-anomaly-detection/src/printer_anomaly

In [24]:
train, test = 0.8, 0.2

In [25]:
# (name, split, before, after)
result = []

for config in configs:
    val_files = []
    test_files = []
    train_files = []

    files = list(sorted(config.files, key=lambda f: f.recording_time, reverse=True))
    total_duration = config.duration

    while sum(map(lambda f: f.duration, test_files)) < total_duration * test:
        test_files.append(files.pop(0))

    train_files = files
    print(f'name={config.name}')
    print(f'train_length={sum(map(lambda f: f.duration, train_files))}')
    print(f'test_length={sum(map(lambda f: f.duration, test_files))}')
    #print(f'val_length={sum(map(lambda f: f.duration, val_files))}')

    result.extend([
        (config.name, 'train', train_files[0].recording_time, train_files[-1].recording_time + 1),
        (config.name, 'test', test_files[0].recording_time, test_files[-1].recording_time + 1),
    #    (config.name, 'val', val_files[0].recording_time, val_files[-1].recording_time + 1)
    ])

name=original
train_length=8568.400000000001
test_length=8283.7
name=exhaust
train_length=55744.40000000001
test_length=25070.800000000003
name=direct_drive
train_length=20763.6
test_length=14356.9


In [26]:
import csv

with open(dataset_path / 'datasets.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['name', 'split', 'before', 'after'])
    writer.writerows(result)

In [27]:
from printer_anomaly_detection.dataset.audio import load_audio_dataset_split, Split

In [10]:
Split.TRAIN.name

'TRAIN'

In [11]:
dataset = load_audio_dataset_split(dataset_path, 'original', Split.TEST, 256, 64)

2023-06-20 23:03:17.874228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-20 23:03:17.889725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-20 23:03:17.889935: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [12]:
from printer_anomaly_detection.dataset.audio import load_audio_dataset_split, Split


In [34]:
dataset = load_audio_dataset_split(dataset_path, 'original', Split.TEST, window_size=256, step_size=32)

In [35]:
len(list(dataset.take(1000)))

2023-06-20 23:08:11.097572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1000

: 